In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

def preprocess_data(folder_path, img_size=(224, 224)):
    """
    Preprocesses image data from a folder containing subfolders for each class.

    Parameters:
    - folder_path: Path to the root folder containing subfolders for each class.
    - img_size: Tuple, size to resize the images to (default is (224, 224)).

    Returns:
    - X: NumPy array containing the preprocessed images.
    - y: NumPy array containing the labels.
    """

    images = []
    labels = []

    for i, class_name in enumerate(os.listdir(folder_path)):
        class_folder = os.path.join(folder_path, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path)

            # Resize image
            img = cv2.resize(img, img_size)

            # Convert to grayscale
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Normalize pixel values (0-1)
            img = img.astype('float32') / 255.0

            images.append(img)
            labels.append(i)  # Assign a label based on folder index

    X = np.array(images)
    y = np.array(labels)

    return X, y

# Example folder structure: 
# - Root folder
#    - autism (contains images of autism children)
#    - normal (contains images of normal children)

root_folder = "datasets/autism"  # Update with your root folder path
img_size = (224, 224)

# Preprocess data
X, y = preprocess_data(root_folder, img_size)

# Shuffle the data
X, y = shuffle(X, y, random_state=42)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for CNN input (add channel dimension for grayscale)
X_train = X_train.reshape(X_train.shape[0], img_size[0], img_size[1], 1)
X_test = X_test.reshape(X_test.shape[0], img_size[0], img_size[1], 1)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Flatten the training and testing data
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Train SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)
svm_classifier.fit(X_train_flat, y_train)

# Make predictions
train_predictions = svm_classifier.predict(X_train_flat)
test_predictions = svm_classifier.predict(X_test_flat)

# Evaluate model
train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Print classification report
print("\nTraining Classification Report:")
print(classification_report(y_train, train_predictions))

print("\nTesting Classification Report:")
print(classification_report(y_test, test_predictions))
